In [1]:
%matplotlib qt
import os
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
from mne.io import read_raw_brainvision, read_raw_fif

from scipy.signal import find_peaks
import numpy as np
from mne.channels import read_dig_captrak, make_standard_montage
from mne.preprocessing import ICA
from mne_icalabel import label_components
import customtkinter as ctk
from mne.coreg import Coregistration
from mne import (open_report, events_from_annotations, Epochs, write_trans,
                setup_source_space, make_bem_model, make_bem_solution, make_forward_solution, compute_covariance,
                extract_label_time_course, read_labels_from_annot)
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import fetch_fsaverage
from scipy.signal import find_peaks
import warnings
from mne.preprocessing import ICA, EOGRegression, create_eog_epochs, create_ecg_epochs, compute_proj_ecg, compute_proj_eog, EOGRegression

In [ ]:
%%latex
##### for preprocessing()
## ask philip where is PO7, PO8, O1 and O2 and adjust it here
## put these cahnnels as misc and see what we can do with it
## kernel crash when running ctk
## ssp for ecg (pulse)
## ssp for vertical eye
## regression for horizontal eye
## .load_data()
## add teas
## GSR, X, Y, Y drop
## rename y-axis in pulse plot maybe *1e9 and time shows
## mne.preprocessing.annotate_break
## interpolate bads (mne.preprocessing.find_bad_channels_lof)



##### for run_analysis()
## in gpias dont do baselining
## double check with repo codes
## ask gpias triggers code
## documenatiaion
## 

<IPython.core.display.Latex object>

In [4]:
# This is a purely internal Zurich function and not needed anymore.
#from files_prepration import create_subjects_dir

#subject_id = "xrtt"
#create_subjects_dir(subject_id=subject_id,
#                        raws_dir="/Volumes/G_USZ_ORL$/Research/ANTINOMICS/data",
#                        subjects_dir="/Volumes/Extreme SSD/payam_data/antinomics_data",
#                        skip_list=["gpias", "xxxxx", "xxxxy", "omi"])

In [2]:
subjects_dir='/tmp/data_in'
subject_id = 'con1'
paradigm = 'rest'
site="Regensburg"

In [ ]:
%matplotlib qt
from eeg.eeg_preprocessing import preprocessing
preprocessing(subject_id=subject_id,
				site=site,
                subjects_dir=subjects_dir,
                paradigm=paradigm,
                manual_data_scroll=True,
                run_ica=False,
                manual_ica_removal=False,
                ssp_eog=False,
                ssp_ecg=False,
                create_report=True,
                saving_dir=None,
                verbose="ERROR")

                                                  
 20%|█████████                                    

Loading raw EEG data ...



                                                  
 20%|█████████                                    

Resampling, filtering and re-referencing ...



In [8]:
%matplotlib qt
from eeg.eeg_rs_processing import run_rs_analysis

run_rs_analysis(
        subject_id=subject_id,
        subjects_dir=subjects_dir,
        visit=1,
        event_ids=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )


ModuleNotFoundError: No module named 'eeg.eeg_rs_processing'

In [ ]:
%matplotlib qt
from eeg_erp_processing import run_erp_analysis

run_erp_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        paradigm="gpias",
        events=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )

Eyetracking

In [ ]:
import mne
raw = mne.io.read_raw_brainvision("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/xxxxx/zdfy_xxxxx.vhdr")
mne.read_events(raw)

In [ ]:
raw.annotations

In [ ]:
from mne.io import read_raw_brainvision, read_raw_eyelink
from mne import events_from_annotations
from mne.preprocessing import realign_raw


## load
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 
events_eeg, event_ids_eeg = events_from_annotations(raw_eeg)
events_eye, event_ids_eye = events_from_annotations(raw_eye)

## for now
stim_id_eeg_1, stim_id_eeg_2 = 8, 9
stim_id_eye_1, stim_id_eye_2 = 4, 5

s_raw = events_eeg[(events_eeg[:,2] == stim_id_eeg_1) | (events_eeg[:,2] == stim_id_eeg_2)][:, 0]
s_other = events_eye[(events_eye[:,2] == stim_id_eye_1) | (events_eye[:,2] == stim_id_eye_2)][:, 0]

## realigning
realign_raw(raw=raw_eeg,
            other=raw_eye,
            t_raw=s_raw / raw_eeg.info["sfreq"] - raw_eeg.first_time,
            t_other=s_other / raw_eye.info["sfreq"] - raw_eye.first_time,
            verbose=None)

raw_eye.add_channels([raw_eeg], force_update_info=True)
events, event_dict = events_from_annotations(raw_eye)
del raw_eeg  

In [ ]:
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 

In [ ]:
raw_eye.plot()

In [ ]:
raw_eye.plot()